# Telegram bot to download multiple Telegram images

In [ ]:
import logging
from telegram import Update, KeyboardButton, ReplyKeyboardMarkup
from telegram.ext import ApplicationBuilder, ContextTypes, CommandHandler, MessageHandler, filters, JobQueue
import nest_asyncio
import urllib.request
import shutil
import os
import datetime as dt

nest_asyncio.apply()

We configure the logging and our bot token.

In [ ]:
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

token = "Private"

Now we define the functions.
* `def start` is when you send "/start" to the bot to activate it.
* `def finish_message` is to download the zip file with all the images once all of them are sent and processed.
* `def remove_job_if_exists` is to avoid sending multiples finish messages, but only one at the end.
* `def image_handler` is the function that is triggered when the user sent a image.
* `def send_zip` send the zip file once the user tap or send "/finish".

In [ ]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await context.bot.send_message(chat_id=update.effective_chat.id, text="I'm a bot, please talk to me!")

async def finish_message(context: ContextTypes.DEFAULT_TYPE):
    await context.bot.send_message(context.job.chat_id, text="Tap /finish or send /finish to the bot in order to get the images.")

def remove_job_if_exists(name: str, context: ContextTypes.DEFAULT_TYPE) -> bool:
    current_jobs = context.job_queue.get_jobs_by_name(name)
    if not current_jobs:
        return False
    for job in current_jobs:
        job.schedule_removal()
    return True

async def image_handler(update: Update, context: ContextTypes.DEFAULT_TYPE):
    image = await update.message.photo[-1].get_file()
    
    if not os.path.exists("C:/Users/Atenea/Desktop/temp"):
        os.mkdir("C:/Users/Atenea/Desktop/temp")

    await image.download_to_drive(f"C:/Users/Atenea/Desktop/temp/{image.file_unique_id}.jpg")

    job_removed = remove_job_if_exists("image", context)
    context.job_queue.run_once(name="image",callback=finish_message, when=update.message.date + dt.timedelta(seconds = 5),chat_id=update.effective_chat.id)

async def send_zip(update: Update,context: ContextTypes.DEFAULT_TYPE):

    shutil.make_archive("C:/Users/Atenea/Desktop/images_downloaded", 'zip', "C:/Users/Atenea/Desktop/temp")
    shutil.rmtree("C:/Users/Atenea/Desktop/temp")
    await context.bot.send_document(chat_id=update.effective_chat.id, document="C:/Users/Atenea/Desktop/images_downloaded.zip")
    shutil.rmtree("C:/Users/Atenea/Desktop/images_downloaded.zip")


Here we define the behaviour of the app and run it.

In [ ]:
if __name__ == '__main__':
    application = ApplicationBuilder().token(token).build()
    
    start_handler = CommandHandler('start', start)
    image_handler = MessageHandler(filters.PHOTO, image_handler)
    send_zip_handler = CommandHandler("finish", send_zip)

    application.add_handler(start_handler)
    application.add_handler(image_handler)
    application.add_handler(send_zip_handler)

    application.run_polling()